In [ ]:
# only run once
!conda create -n resnet_fl_2host python=3 anaconda -y

In [1]:
# activate conda environment to access pysyft
!source /usr/local/anaconda3/bin/activate resnet_fl_2host

In [3]:
# not necessary after 2nd run
!conda install -c pytorch pytorch -y

Solving environment: done

# All requested packages already installed.



In [ ]:
# not necessary after 2nd run
!pip install syft==0.2.9

In [1]:
import sys
import torch
#torch.set_default_tensor_type(torch.cuda.FloatTensor)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import syft as sy

In [2]:
hook = sy.TorchHook(torch)
host1 = sy.VirtualWorker(hook, id="host1")
host2 = sy.VirtualWorker(hook, id="host2")
data_dir = "UCMerced_LandUse/Images"

In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 1
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = True

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()
print(use_cuda)
torch.manual_seed(args.seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print(kwargs)

False
cpu
{}


In [4]:
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    loaded_dataset = datasets.ImageFolder(data_dir, transform=train_transforms)
    
    num_train = len(loaded_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)

    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    federated_train_loader = sy.FederatedDataLoader(
        loaded_dataset.federate((host1, host2)),
        batch_size=args.batch_size,
        sampler=train_sampler,
        #shuffle=True, 
        **kwargs
    )
    train_loader = torch.utils.data.DataLoader(
        loaded_dataset,
        batch_size=args.batch_size,
        sampler=train_sampler,
        #shuffle=True, 
        **kwargs
    )
    test_loader = torch.utils.data.DataLoader(
        loaded_dataset,
        batch_size=args.test_batch_size,
        sampler=test_sampler,       
        #shuffle=True,
        **kwargs
    )
    
    return {'train': federated_train_loader, 'val': test_loader}, {'train': len(train_idx), 'val':len(test_idx)}

In [5]:
dataloaders, dataset_sizes = load_split_train_test(data_dir, .2)
print(dataloaders['val'].dataset.classes, dataset_sizes)
class_names = dataloaders['val'].dataset.classes

['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings', 'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse', 'harbor', 'intersection', 'mediumresidential', 'mobilehomepark', 'overpass', 'parkinglot', 'river', 'runway', 'sparseresidential', 'storagetanks', 'tenniscourt'] {'train': 1680, 'val': 420}


In [ ]:
torch.set_num_threads(4)

In [ ]:
'''
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))
'''
'''
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    '''

In [6]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=5, batch_size=4):
    %matplotlib inline
    import pylab as pl
    from IPython import display
    
    def live_plot(data):
        pl.plot(data)
        display.clear_output(wait=True)
        display.display(pl.gcf())
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    loss_values = []

    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        model.train()  # Set model to training mode
        running_loss_train, running_loss_val = 0, 0
        running_corrects_train, running_corrects_val = 0, 0

        # Iterate over data.
        for inputs, labels in tqdm(dataloaders['train']):
            # NEW) send model to correct worker
            model.send(inputs.location)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                # backward + optimize only if in training phase

                loss.backward()
                optimizer.step()


            # get model (with gradients)
            model.get()
            # statistics
            running_loss_train += loss.get().item() * batch_size
            running_corrects_train += torch.sum(preds == labels.data).get()
            #running_loss_train += loss.item() * batch_size
            #running_corrects_train += torch.sum(preds == labels.data)
        scheduler.step()
        
        epoch_loss_train = running_loss_train / dataset_sizes['train']
        epoch_acc_train = running_corrects_train.double() / dataset_sizes['train']
        loss_values.append(epoch_loss_train)
        live_plot(loss_values)
        print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(epoch_loss_train, epoch_acc_train))

        model.eval()   # Set model to evaluate mode
        for inputs, labels in tqdm(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            running_loss_val += loss.item() * batch_size
            running_corrects_val += torch.sum(preds == labels.data)
        epoch_loss_val = running_loss_val / dataset_sizes['val']
        epoch_acc_val = running_corrects_val.double() / dataset_sizes['val']
        print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss_val, epoch_acc_val))

        # deep copy the model
        if epoch_acc_val > best_acc:
            best_acc = epoch_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
class Net(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        resnet = models.resnet34(pretrained=True)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(num_ftrs, n_classes)
        )
        self.base_model = resnet
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        return self.sigm(self.base_model(x))

In [8]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        from collections import OrderedDict
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(5, 5))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(5, 5))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(5, 5))
        self.linear1 = nn.Linear(64 * 24 * 24, 120)
        self.linear2 = nn.Linear(120, 84)
        self.linear3 = nn.Linear(84, n_classes)                                
    def forward(self, x):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv3(x)), kernel_size=2, stride=2)
        x = x.view(-1, 64 * 24 * 24)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [9]:
from torchvision import models
from torch.optim import lr_scheduler
import time
import os
import copy

model = CNN(len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, num_epochs=20, batch_size=args.batch_size)
torch.save(model.state_dict(), "resnet_fl2.pt")

Epoch 0/19
----------


In [32]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer_ft, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "resnet_fl.pt")

Train Epoch: 1 [0/2112 (0%)]	Loss: -0.493185
Train Epoch: 1 [1920/2112 (91%)]	Loss: -0.518956

Test set: Average loss: -0.5051, Accuracy: 120/2100 (6%)

Train Epoch: 2 [0/2112 (0%)]	Loss: -0.488512
Train Epoch: 2 [1920/2112 (91%)]	Loss: -0.496494

Test set: Average loss: -0.5063, Accuracy: 101/2100 (5%)

Train Epoch: 3 [0/2112 (0%)]	Loss: -0.514812
Train Epoch: 3 [1920/2112 (91%)]	Loss: -0.513043

Test set: Average loss: -0.5055, Accuracy: 97/2100 (5%)

Train Epoch: 4 [0/2112 (0%)]	Loss: -0.508560
Train Epoch: 4 [1920/2112 (91%)]	Loss: -0.487546

Test set: Average loss: -0.5041, Accuracy: 98/2100 (5%)

Train Epoch: 5 [0/2112 (0%)]	Loss: -0.516424
Train Epoch: 5 [1920/2112 (91%)]	Loss: -0.531590

Test set: Average loss: -0.5080, Accuracy: 101/2100 (5%)

Train Epoch: 6 [0/2112 (0%)]	Loss: -0.474073
Train Epoch: 6 [1920/2112 (91%)]	Loss: -0.528637

Test set: Average loss: -0.5071, Accuracy: 101/2100 (5%)

Train Epoch: 7 [0/2112 (0%)]	Loss: -0.516183
Train Epoch: 7 [1920/2112 (91%)]	Loss: 